In [19]:
import pickle
from model import Node, categorize_next, display_lazy_tree, format_node, format_time

In [20]:
import chardet

with open("amazon_products.csv", 'rb') as f:
    result = chardet.detect(f.read(8*1024*1024))
    print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [21]:
import pandas as pd
df = pd.read_csv("amazon_products.csv", encoding=result["encoding"])

In [22]:
df = df.sample(20000)
df.head()

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
1206274,B00JY090KS,"Daystar MX30013RE Mud Scraper, Red",https://m.media-amazon.com/images/I/51lJm81a5U...,https://www.amazon.com/dp/B00JY090KS,4.1,0,23.28,0.0,19,False,0
110582,B083XYM53Z,Fielect 3Pcs Carbon Fiber Strip Bars 2x10x600m...,https://m.media-amazon.com/images/I/51MO9GN3sX...,https://www.amazon.com/dp/B083XYM53Z,4.8,9,21.89,0.0,140,False,0
369706,B0C1Z8866Y,Arcade Game Console Pandora Box Built in 10000...,https://m.media-amazon.com/images/I/81XWHRgx+5...,https://www.amazon.com/dp/B0C1Z8866Y,3.4,0,109.99,0.0,222,False,0
1019835,B0BM735RF5,"Wood Cutting Boards for Kitchen, Large Cutting...",https://m.media-amazon.com/images/I/71MeZTjGwg...,https://www.amazon.com/dp/B0BM735RF5,4.5,0,39.99,0.0,170,False,800
110404,B09YQ13MC2,Swpeet 15Pcs Milkly Plastic Rods Assortment Ki...,https://m.media-amazon.com/images/I/51lrTdtxV5...,https://www.amazon.com/dp/B09YQ13MC2,4.7,4,7.98,0.0,140,False,0


In [23]:
with open("./tree_v3_improved.pkl", "rb") as f:
    root = pickle.load(f)

In [24]:
from typing import Callable, Optional, Tuple
from langchain_core.language_models.chat_models import BaseChatModel

from model.data import TokenCounts

def classify(item_description: str, root: Node, create_llm: Callable[[], BaseChatModel]) -> Tuple[Optional[Node], TokenCounts]:
    def classify_recur(item_description: str, node: Node) -> Tuple[Optional[Node], TokenCounts]:
        if node.is_leaf():
            return node, TokenCounts()
        children = [*node.children]
        all_tokens = TokenCounts()
        print(f"Entering {node.condition}")
        while len(children) > 0:
            print(f"Trying {node.condition} with {len(children)} unexplored child nodes.")
            choice, tokens = categorize_next(item_description=item_description, nodes=children, create_llm=create_llm)
            print(f"Chose {choice.condition if choice else "None"}")
            all_tokens += tokens
            if choice is None:
                print(f"Retrying previous node with mask.")
                return (None, all_tokens)
            
            print(f"Trying chosen subtree.")
            result, tokens = classify_recur(item_description=item_description, node=choice)
            print(f"Subtree yielded choice {result.condition if result else "None"}.")
            all_tokens += tokens
            if result is not None:
                print(f"Yielding subtree result {result.condition}")
                return (result, all_tokens)
            
            print(f"Retrying {node.condition} with mask.")
            children.remove(choice)
            
        return None, all_tokens
            
    return classify_recur(item_description=item_description, node=root)

In [25]:
from langchain_ollama import ChatOllama
create_llm = lambda: ChatOllama(
    model="qwen2.5:32b",
    # temperature=0,
)

In [26]:
display_lazy_tree(root, max_initial_depth=5)

    'data': [{'branchvalues': 'total',
              'ids': [5c070ee8-39ba-49d4-…

In [27]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_community.callbacks import get_openai_callback


class Description(BaseModel):
    reasoning: str = Field(description="Reasoning on your choice.")
    description: str = Field(description="The shortened description")

def prompt_tentative_description(item_description: str, create_llm: Callable[[], BaseChatModel]) -> Tuple[Description, TokenCounts]:
    template = """
    From the given e-commerce item description give a simplified short, generic description that would be useful in categorizing an item into product standardizations.
    From specific product names and branding provide a description that includes the true nature of the product.
    Use the given tool to provide your answer.
    
    {item}
    """.strip()
    
    prompt = ChatPromptTemplate.from_template(template)
    prompt = prompt.format(item=item_description)
    
    with get_openai_callback() as cb:
        llm = create_llm().with_structured_output(Description)
        response = llm.invoke([HumanMessage(prompt)])
        
    return response, TokenCounts(prompt=cb.prompt_tokens, completion=cb.completion_tokens, total=cb.total_tokens)
    

In [28]:
from ipywidgets import IntProgress, VBox, Label
from IPython.display import display
import time

def create_classification_sample(df: pd.DataFrame, root: Node) -> Tuple[pd.DataFrame, TokenCounts]:
    items: list[dict[str, any]] = []
    all_tokens = TokenCounts()

    total_rows = len(df)
    progress_bar = IntProgress(min=0, max=total_rows, description='Progress:', bar_style='info')
    label = Label(value=f"0/{total_rows} items processed")
    time_label = Label(value="Estimating time...")
    display(VBox([progress_bar, label, time_label]))

    processed_rows = 0
    start_time = time.time()

    for i, row in df.iterrows():
        print(f"\tItem: {row['title']}")
        short_desc, desc_tokens = prompt_tentative_description(item_description=row["title"], create_llm=create_llm)
        print(f"\tShort Desc: {short_desc.description}")
        node, tokens = classify(item_description=short_desc.description, root=root, create_llm=create_llm)
        record = {
            "item": row["title"],
            "imputed_desc": short_desc.description,
            "classification": format_node(node) if node else "None"
        }
        items.append(record)
        all_tokens += tokens + desc_tokens

        processed_rows += 1
        elapsed_time = time.time() - start_time
        progress_bar.value = processed_rows
        label.value = f"{processed_rows}/{total_rows} items processed"
        estimated_total_time = elapsed_time / processed_rows * total_rows
        remaining_time = estimated_total_time - elapsed_time
        time_label.value = f"Estimated remaining time: {format_time(remaining_time)}"

    result = pd.DataFrame(items)
    return result, all_tokens


In [ ]:
df_sample = df.sample(100)

args: list[dict[str,any]] = [
    # {
    #     "model_file": "tree_v3.pkl",
    #     "output_file": "classifications_v3.csv"
    # },
    {
        "model_file": "tree_v3_improved.pkl",
        "output_file": "classifications_v3_improved.csv"
    }
]

output_dir = "classification_results"
tokens_output_file = "tokens.csv"

import os
import time

os.makedirs(output_dir, exist_ok=True)


tokens_dicts = []
for argset in args:
    start_time = time.time()
    with open(argset["model_file"], "rb") as f:
        root = pickle.load(f)
    
    out_df, tokens = create_classification_sample(df_sample, root)
    elapsed_time = time.time() - start_time
    tokens_dicts.append({
        **argset,
        "prompt": tokens.prompt,
        "completion": tokens.completion,
        "total": tokens.total,
        "elapsed_seconds": elapsed_time
    })
    out_df.to_csv(argset["output_file"])
    print(f"Saved classifications from {argset['model_file']} to {output_dir}/{argset['output_file']}")
    
tokens_df = pd.DataFrame(tokens_dicts)
tokens_df.to_csv(tokens_output_file)

	Item: Luminara Moving Flame LED Flameless Tealight (1.4" x 2") Remote Ready Battery Operated Plastic LED Flameless Tea Light - Smooth Matte - Pearl Ivory (4-Pack)
	Short Desc: Flameless LED tea light - battery operated plastic candle alternative in pearl ivory color (pack of 4)
Entering All Products
Trying All Products with 20 unexplored child nodes.
Chose HomeAutomationAndDecor
Trying chosen subtree.
Entering HomeAutomationAndDecor
Trying HomeAutomationAndDecor with 25 unexplored child nodes.
Chose Home Decor & Crafting Supplies
Trying chosen subtree.
Entering Home Decor & Crafting Supplies
Trying Home Decor & Crafting Supplies with 5 unexplored child nodes.
Chose None
Retrying previous node with mask.
Subtree yielded choice None.
Retrying HomeAutomationAndDecor with mask.
Trying HomeAutomationAndDecor with 24 unexplored child nodes.
Chose Seasonal Décor
Trying chosen subtree.
Subtree yielded choice Seasonal Décor.
Yielding subtree result Seasonal Décor
Subtree yielded choice Seasona

KeyboardInterrupt: 

: 